In [1]:
import json, pathlib
from collections import defaultdict

NEW_COMPLAINT_TYPES = {
    "category",
    "appearance",
    "handle",
    "collision",
    "joint",
    "metalink",
    "unknown",
}

complaints_by_model = defaultdict(list)
complaints_by_type = defaultdict(list)
for path in pathlib.Path(r"D:\ig_pipeline").glob("qa-2025/*/*/*.json"):
    try:
        batch = path.parts[-3]
        mdl = path.stem
        complaints = json.loads(path.read_text())["complaints"]

        # Update all of the QC: complaints to match the author's batch
        for c in complaints:
            c_question = c["message"]
            if c_question.startswith("QC: "):
                c["complaint"] = f"quick complaint added during qa-2025 batch {batch}"

        unresolved_complaints = [x for x in complaints if not x["processed"]]
        complaints_by_model[mdl].extend(unresolved_complaints)

        for c in unresolved_complaints:
            c_question = c["message"]
            if c_question.startswith("QC: "):
                c_type = c_question[4:]
                assert c_type in NEW_COMPLAINT_TYPES
            else:
                c_type = "legacy"
            complaints_by_type[c_type].append(c)
    except:
        print(f"Error processing {path}")

In [2]:
with open(r"D:\ig_pipeline\artifacts\pipeline\object_inventory.json", "r") as f:
    providers = {x.split("-")[1]: y for x, y in json.load(f)["providers"].items()}

In [5]:
import collections

complaints_by_target = collections.defaultdict(list)
for obj, complaints in complaints_by_model.items():
  complaints_by_target[providers[obj]].extend(complaints)

In [6]:
complaints_by_target

defaultdict(list,
            {'objects/task_relevant-xy': [{'object': 'bag_of_rubbish-qfkzkd',
               'message': 'QC: appearance',
               'complaint': 'quick complaint added during qa-2025 batch qa-2025-eric',
               'processed': False,
               'new': True},
              {'object': 'hutch-vsqsuy',
               'message': 'QC: collision',
               'complaint': 'quick complaint added during qa-2025 batch qa-2025-eric',
               'processed': False,
               'new': True},
              {'object': 'lid-bemtjd',
               'message': 'QC: category',
               'complaint': 'quick complaint added during qa-2025 batch qa-2025-eric',
               'processed': False,
               'new': True},
              {'object': 'lid-snklci',
               'message': 'QC: category',
               'complaint': 'quick complaint added during qa-2025 batch qa-2025-eric',
               'processed': False,
               'new': True},
          

In [7]:
# Save all the complaints
import os
for target, complaints in complaints_by_target.items():
    with open(os.path.join(r"D:\ig_pipeline\cad", *(target.split("/")), "complaints.json"), "w") as f:
        json.dump(complaints, f, indent=4)